min. x1-x2+x3  
s.t. x1>=1, x2>=1, x3>=2,  
2x1 − x2 + 2x3 ≤ 4,  
2x1 − 3x2 + x3 ≤ −5,  
−x1 + x2 − 2x3 ≤ −1.  


In [9]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc


In [10]:
from pyomo.environ import *

import numpy as np

model=ConcreteModel()

N=3
M=3

obj_coef=np.array([1,-1,1])

constr_coef=np.array([[2,-1,2],
                     [2,-3,1],
                     [-1,1,-2]])



constraint_rhs=np.array([4,-5,-1])

lower_bounds=np.array([1,1,2])
upper_bounds=np.array([np.inf,np.inf,np.inf])

col_indices=np.arange(N)
row_indices=np.arange(M)

model.x=Var(col_indices)

for i in col_indices:
    model.x[i].setlb(lower_bounds[i])
    model.x[i].setub(upper_bounds[i])


model.constraint=ConstraintList()

for i in row_indices:
    model.constraint.add(sum(model.x[j]*constr_coef[i,j] for j in col_indices)<=constraint_rhs[i])

model.objective=Objective(expr=summation(obj_coef,model.x),sense=minimize)

model.pprint()

cbc=SolverFactory('cbc')
glpk=SolverFactory('glpk', executable='/usr/bin/glpsol')


2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     1 :  None :   inf : False :  True :  Reals
          2 :     2 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - x[1] + x[2]

1 Constraint Declarations
    constraint : Size=3, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] - x[1] + 2*x[2] :   4.0 :   True


In [11]:
result_cbc=cbc.solve(model)
print("cbcresult",result_cbc)
print("Solver Status:",result_cbc.solver.status)
print("Solver Termaination Condition:",result_cbc.solver.termination_condition)

    model=unknown;
        message from solver=<undefined>
cbcresult 
Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: unbounded
  Termination message: Model was proven to be unbounded.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.021604061126708984

Solver Status: warning
Solver Termaination Condition: unbounded


On using coinor-cbc solver :
Solver Status: warning 
Sover Termination Condition: unbounded

The messages received above are quite clear. It is clearly mentioned that the problem is unbounded so it is aproaching -infinity in this case and hence the status shown is warning which is quite justified.  
This happens because cbc is designed to give clear messages in these situations and the messages are nicely interpreted by pyomo.

In [12]:
result_glpk=glpk.solve(model)
print("glpkresult",result_glpk)
print("Solver Status:",result_glpk.solver.status)
print("Solver Termaination Condition:",result_glpk.solver.termination_condition)

glpkresult 
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.015077590942382812

Solver Status: ok
Solver Termaination Condition: other


On using glpk solver, the solver status is ok and termination condition is other. These messages do not give us very meaningful information about what is happening in the problem but one thing is clear that the problem is not solved to optimum otherwise termination condition would be optimal. But the Solver Status being ok does not make any sense here.

The possible reasons for such ambiguous message might be that the glpk is not designed to give very clear message in such situations and its messages are not interpreted properly by pyomo.

In [13]:
#1st change:changing optimization sense
model.objective.set_sense(maximize)
model.pprint()

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     1 :  None :   inf : False :  True :  Reals
          2 :     2 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x[0] - x[1] + x[2]

1 Constraint Declarations
    constraint : Size=3, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] - x[1] + 2*x[2] :   4.0 :   True


In [14]:
result_cbc=cbc.solve(model)
print(result_cbc)
print("solver status",result_cbc.solver.status)
print("solver termination condition",result_cbc.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 0.4
  Upper bound: 0.4
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.021485567092895508
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

solver status ok
solver termination condition optimal


Solver Status is ok and termination condition is optimal. This indicates that the problem was solved and optimal condition is reached and hence the proper messages. 

In [17]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraint.display()


Objective =  0.3999999999999999

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  3.2
x[ 2 ] =  2.6

Constraints
constraint : Size=3
    Key : Lower : Body               : Upper
      1 :  None :                4.0 :   4.0
      2 :  None : -5.000000000000002 :  -5.0
      3 :  None :               -3.0 :  -1.0


Constraint 1 and 2 are active.  
Constraint 3 is inactive.

In [18]:
# 2nd change: setting upper bound on variable 2
model.objective.set_sense(minimize)
model.x[1].setub(5)
model.pprint()
result_cbc=cbc.solve(model)
print(result_cbc)
print("solver status",result_cbc.solver.status)
print("solver termination condition",result_cbc.solver.termination_condition)

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :   1.0 :   inf : False : False :  Reals
          1 :     1 :   3.2 :     5 : False : False :  Reals
          2 :     2 :   2.6 :   inf : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - x[1] + x[2]

1 Constraint Declarations
    constraint : Size=3, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] - x[1] + 2*x[2] :   4.0 :   True


Solver Status is ok and termination condition is optimal. This indicates that the problem was solved and optimal condition is reached and hence the proper messages. 

In [19]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraint.display()


Objective =  -1.5

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  5.0
x[ 2 ] =  2.5

Constraints
constraint : Size=3
    Key : Lower : Body  : Upper
      1 :  None :   2.0 :   4.0
      2 :  None : -10.5 :  -5.0
      3 :  None :  -1.0 :  -1.0


Constraint 1 and 2 are inactive.  
Constraint 3 is active

In [20]:
# 3rd change: adding another constraint 
model.x[1].setub(np.inf)
new_constr_coef = [0,1,-1]
new_constr_rhs = 6
model.constraint.add(sum(new_constr_coef[j]*model.x[j] for j in col_indices) <= new_constr_rhs)
model.pprint()
result_cbc=cbc.solve(model)
print(result_cbc)
print("solver status",result_cbc.solver.status)
print("solver termination condition",result_cbc.solver.termination_condition)


2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :   1.0 :   inf : False : False :  Reals
          1 :     1 :   5.0 :   inf : False : False :  Reals
          2 :     2 :   2.5 :   inf : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - x[1] + x[2]

1 Constraint Declarations
    constraint : Size=4, Index=constraint_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] - x[1] + 2*x[2] :   4.0 :   Tr

Solver Status is ok and termination condition is optimal. This indicates that the problem was solved and optimal condition is reached and hence the proper messages. 

In [21]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraint.display()


Objective =  -5.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  12.0
x[ 2 ] =  6.0

Constraints
constraint : Size=4
    Key : Lower : Body  : Upper
      1 :  None :   2.0 :   4.0
      2 :  None : -28.0 :  -5.0
      3 :  None :  -1.0 :  -1.0
      4 :  None :   6.0 :   6.0


Constraint 1 and 2 are inactive.  
Constraint 3 and 4 are active.

In [22]:
#4th change: changing objective and one of the constraints

model.constraint[4].deactivate()

model.constraint[2].deactivate()
model.constraint.add(expr=model.x[0]+model.x[1]>=25)

model.objective.deactivate()
model.new_objective=Objective(expr=model.x[0]+model.x[1]+model.x[2],sense=minimize)


model.pprint()

result_cbc=cbc.solve(model)
print(result_cbc)
print("solver status",result_cbc.solver.status)
print("solver termination condition",result_cbc.solver.termination_condition)



2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :   1.0 :   inf : False : False :  Reals
          1 :     1 :  12.0 :   inf : False : False :  Reals
          2 :     2 :   6.0 :   inf : False : False :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + x[1] + x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : x[0] - x[1] + x[2]

1 Constraint Declarations
    constraint : Size=5, I

Solver Status is ok and termination condition is optimal. This indicates that the problem was solved and optimal condition is reached and hence the proper messages. 

In [24]:
print('\nObjective = ', model.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraint.display()


Objective =  35.0

Decision Variables
x[ 0 ] =  3.0
x[ 1 ] =  22.0
x[ 2 ] =  10.0

Constraints
constraint : Size=5
    Key : Lower : Body : Upper
      1 :  None :  4.0 :   4.0
      3 :  None : -1.0 :  -1.0
      5 :  25.0 : 25.0 :  None


All the 3 constraints are active.